# Classification with PySpark

In [1]:
# Load Library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from pyspark.sql import SparkSession, SQLContext
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean, col, split, regexp_extract, when, isnan, count, create_map, lit
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from itertools import chain

In [2]:
# Create Spark Session (like a container)
spark = SparkSession.builder.appName('PySpark with ML').getOrCreate()
train_df = spark.read.csv('train.csv', header=True, inferSchema=True)
test_df = spark.read.csv('test.csv', header=True, inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/02 17:29:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
train_df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [4]:
train_df.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [5]:
train_df.limit(5).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [6]:
train_df.select('Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare').summary().show()

+-------+-------------------+------------------+------------------+------------------+-------------------+-----------------+
|summary|           Survived|            Pclass|               Age|             SibSp|              Parch|             Fare|
+-------+-------------------+------------------+------------------+------------------+-------------------+-----------------+
|  count|                891|               891|               714|               891|                891|              891|
|   mean| 0.3838383838383838| 2.308641975308642| 29.69911764705882|0.5230078563411896|0.38159371492704824| 32.2042079685746|
| stddev|0.48659245426485753|0.8360712409770491|14.526497332334035|1.1027434322934315| 0.8060572211299488|49.69342859718089|
|    min|                  0|                 1|              0.42|                 0|                  0|              0.0|
|    25%|                  0|                 2|              20.0|                 0|                  0|           7.8958|


## EDA

In [7]:
train_df.groupBy('Survived').count().show()

+--------+-----+
|Survived|count|
+--------+-----+
|       1|  342|
|       0|  549|
+--------+-----+



In [8]:
train_df.groupBy('Survived').mean('Age', 'Fare').show()

+--------+------------------+------------------+
|Survived|          avg(Age)|         avg(Fare)|
+--------+------------------+------------------+
|       1|28.343689655172415| 48.39540760233917|
|       0| 30.62617924528302|22.117886885245877|
+--------+------------------+------------------+



In [9]:
train_df.groupBy('Survived').pivot('Pclass').count().show()

+--------+---+---+---+
|Survived|  1|  2|  3|
+--------+---+---+---+
|       1|136| 87|119|
|       0| 80| 97|372|
+--------+---+---+---+



In [10]:
train_df.groupBy('Survived').pivot('Sex').count().show()

+--------+------+----+
|Survived|female|male|
+--------+------+----+
|       1|   233| 109|
|       0|    81| 468|
+--------+------+----+



In [11]:
# Those with smaller families had a higher chance of surviving
train_df.groupBy('Survived').pivot('Parch').count().show()

+--------+---+---+---+---+----+---+----+
|Survived|  0|  1|  2|  3|   4|  5|   6|
+--------+---+---+---+---+----+---+----+
|       1|233| 65| 40|  3|null|  1|null|
|       0|445| 53| 40|  2|   4|  4|   1|
+--------+---+---+---+---+----+---+----+



In [12]:
# Those without a partner to travel with were more likely to survive
train_df.groupBy('Survived').pivot('SibSp').count().show()

+--------+---+---+---+---+---+----+----+
|Survived|  0|  1|  2|  3|  4|   5|   8|
+--------+---+---+---+---+---+----+----+
|       1|210|112| 13|  4|  3|null|null|
|       0|398| 97| 15| 12| 15|   5|   7|
+--------+---+---+---+---+---+----+----+



In [13]:
train_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train_df.columns]).show()

+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|PassengerId|Survived|Pclass|Name|Sex|Age|SibSp|Parch|Ticket|Fare|Cabin|Embarked|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+
|          0|       0|     0|   0|  0|177|    0|    0|     0|   0|  687|       2|
+-----------+--------+------+----+---+---+-----+-----+------+----+-----+--------+



In [14]:
train_df.select(count(train_df.Cabin)).show()

+------------+
|count(Cabin)|
+------------+
|         204|
+------------+



Since Cabin has about 77% data missing, and Cabin and Pclass are similar, we can drop Cabin. 

In [15]:
train_df = train_df.drop('Cabin')

In [16]:
train_df.groupBy('Embarked').count().show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       Q|   77|
|    null|    2|
|       C|  168|
|       S|  644|
+--------+-----+



In [17]:
train_df.select('Fare').summary('50%').show()

+-------+-------+
|summary|   Fare|
+-------+-------+
|    50%|14.4542|
+-------+-------+



Since S is the majority of the Embarked column, we will replace the two null values with S. There are missing values in test data for Fare so we will fill in those missing values with the median value.

In [18]:
train_df = train_df.fillna({'Embarked':'S', 'Fare': 14.45})

We will group similar titles together and assign the average age to the missing values for that title. 

In [19]:
train_df = train_df.withColumn('Title', regexp_extract(train_df['Name'], '([A-Za-z]+)\.', 1))
train_df.groupBy('Title').agg(count('Age'), mean('Age')).sort(col('count(Age)').desc()).show()

+--------+----------+------------------+
|   Title|count(Age)|          avg(Age)|
+--------+----------+------------------+
|      Mr|       398|32.368090452261306|
|    Miss|       146|21.773972602739725|
|     Mrs|       108|35.898148148148145|
|  Master|        36| 4.574166666666667|
|     Rev|         6|43.166666666666664|
|      Dr|         6|              42.0|
|     Col|         2|              58.0|
|    Mlle|         2|              24.0|
|   Major|         2|              48.5|
|     Don|         1|              40.0|
|Countess|         1|              33.0|
|    Lady|         1|              48.0|
|Jonkheer|         1|              38.0|
|     Mme|         1|              24.0|
|    Capt|         1|              70.0|
|      Ms|         1|              28.0|
|     Sir|         1|              49.0|
+--------+----------+------------------+



The top 3 titles are Mr, Miss, and Mrs which account for most of the titles of passengers. Master is lower in count than the top three, however, the average age is much lower which could account for a different group of passengers. Thus, we will map the other titles to the top 4 titles.

In [20]:
title_dictionary = {'Mr':'Mr', 'Miss':'Miss', 'Mrs':'Mrs','Master':'Master','Rev':'Mr', 'Dr':'Mr', 'Col':'Mr', 'Mlle':'Miss', 'Major': 'Mr', 'Don':'Mr', 'Countess':'Mrs', 'Lady': 'Mrs', 'Jonkheer':'Mr','Mme':'Miss', 'Capt':'Mr','Ms':'Miss','Sir':'Mr'}
title_map = create_map([lit(x) for x in chain(*title_dictionary.items())])
train_df = train_df.withColumn('Title', title_map[train_df['Title']])
train_df.groupBy('Title').mean('Age').show()

+------+------------------+
| Title|          avg(Age)|
+------+------------------+
|  Miss|             21.86|
|Master| 4.574166666666667|
|    Mr| 33.02272727272727|
|   Mrs|35.981818181818184|
+------+------------------+



In [21]:
def age_imputer(df, title, age):
    return df.withColumn('Age', when((df['Age'].isNull()) & (df['Title'] == title), age).otherwise(df['Age']))

In [22]:
train_df = age_imputer(train_df, 'Mr', 33.02)
train_df = age_imputer(train_df, 'Miss', 21.86)
train_df = age_imputer(train_df, 'Mrs', 35.98)
train_df = age_imputer(train_df, 'Master', 4.57)

Will create a new column named FamilySize to have a count of total members of a family using columns Parch and SibSpl.

In [23]:
train_df = train_df.withColumn('FamilySize', train_df['Parch'] + train_df['SibSp']).drop('Parch', 'SibSp')

We are also dropping other columns we do not need for this analysis.

In [24]:
train_df = train_df.drop('PassengerID', 'Name', 'Ticket', 'Title')

In [25]:
train_df.show(5)

+--------+------+------+----+-------+--------+----------+
|Survived|Pclass|   Sex| Age|   Fare|Embarked|FamilySize|
+--------+------+------+----+-------+--------+----------+
|       0|     3|  male|22.0|   7.25|       S|         1|
|       1|     1|female|38.0|71.2833|       C|         1|
|       1|     3|female|26.0|  7.925|       S|         0|
|       1|     1|female|35.0|   53.1|       S|         1|
|       0|     3|  male|35.0|   8.05|       S|         0|
+--------+------+------+----+-------+--------+----------+
only showing top 5 rows



In [26]:
train_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train_df.columns]).show()

+--------+------+---+---+----+--------+----------+
|Survived|Pclass|Sex|Age|Fare|Embarked|FamilySize|
+--------+------+---+---+----+--------+----------+
|       0|     0|  0|  0|   0|       0|         0|
+--------+------+---+---+----+--------+----------+



## Modeling

We will first convert the 'Sex' and 'Embarked' columns from string to numeric index. 

In [27]:
strInd = StringIndexer(inputCols=['Sex', 'Embarked'], outputCols=['SexNum', 'EmbarkedNum'])
strInd_mod = strInd.fit(train_df)

train_df_new = strInd_mod.transform(train_df).drop('Sex', 'Embarked')
train_df_new.show(5)

+--------+------+----+-------+----------+------+-----------+
|Survived|Pclass| Age|   Fare|FamilySize|SexNum|EmbarkedNum|
+--------+------+----+-------+----------+------+-----------+
|       0|     3|22.0|   7.25|         1|   0.0|        0.0|
|       1|     1|38.0|71.2833|         1|   1.0|        1.0|
|       1|     3|26.0|  7.925|         0|   1.0|        0.0|
|       1|     1|35.0|   53.1|         1|   1.0|        0.0|
|       0|     3|35.0|   8.05|         0|   0.0|        0.0|
+--------+------+----+-------+----------+------+-----------+
only showing top 5 rows



We are going to use VectorAssembler because in scikit learn, it takes X and y in a separation matrix. Usually, y is a column vector and X is a matrix. But for Spark API, X and y has to be in a single matrix instead of two for the training data. It only accepts X in the prediction part. X should also be a vector in each row of the dataframe. We cannot directly feed the dataframe to the model.

In [28]:
vec_assemble = VectorAssembler(inputCols=train_df_new.columns[1:], outputCol='features')
train_df_new = vec_assemble.transform(train_df_new).select('features', 'Survived')
train_df_new.show(5, truncate=False)

+------------------------------+--------+
|features                      |Survived|
+------------------------------+--------+
|[3.0,22.0,7.25,1.0,0.0,0.0]   |0       |
|[1.0,38.0,71.2833,1.0,1.0,1.0]|1       |
|[3.0,26.0,7.925,0.0,1.0,0.0]  |1       |
|[1.0,35.0,53.1,1.0,1.0,0.0]   |1       |
|[3.0,35.0,8.05,0.0,0.0,0.0]   |0       |
+------------------------------+--------+
only showing top 5 rows



In [31]:
# Split data into training and validation first. Will use the test dataset later for prediction results
train_df, validation_df = train_df_new.randomSplit([0.8, 0.2], seed = 0)

In [32]:
train_df.show(5, truncate=False)

+---------------------+--------+
|features             |Survived|
+---------------------+--------+
|(6,[0,1],[1.0,33.02])|0       |
|(6,[0,1],[1.0,33.02])|0       |
|(6,[0,1],[1.0,38.0]) |0       |
|(6,[0,1],[1.0,39.0]) |0       |
|(6,[0,1],[1.0,40.0]) |0       |
+---------------------+--------+
only showing top 5 rows

